In [1]:
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm

In [2]:
data_dir = Path("/Users/keltim01/Documents/data/nfl-big-data-bowl-2023")

In [4]:
files = [x for x in os.listdir(data_dir) if x.startswith("week")]

In [5]:
files

['week1.csv',
 'week2.csv',
 'week3.csv',
 'week7.csv',
 'week6.csv',
 'week4.csv',
 'week5.csv',
 'week8.csv']

In [10]:
df_weekly_data = pd.DataFrame()

# Standardize Data if you haven't written a file or don't want to
for file in tqdm(files):
    df_wk = pd.read_csv(data_dir.joinpath(file))
    
    df_wk['x'] = df_wk.apply(lambda row: 120 - row['x'] if row['playDirection'] == 'left' else row['x'], axis=1)
    df_wk['y'] = df_wk.apply(lambda row: 160 / 3 - row['y'] if row['playDirection'] == 'left' else row['y'], axis=1)
    df_wk['dir'] = df_wk.apply(lambda row: row['dir'] + 180 if row['playDirection'] == 'left' else row['dir'], axis=1)
    df_wk['dir'] = df_wk.apply(lambda row: row['dir'] - 360 if row['dir'] > 360 else row['dir'], axis=1)
    df_wk['o'] = df_wk.apply(lambda row: row['o'] + 180 if row['playDirection'] == 'left' else row['o'], axis=1)
    df_wk['o'] = df_wk.apply(lambda row: row['o'] - 360 if row['o'] > 360 else row['o'], axis=1)
    df_wk['second'] = df_wk['frameId'] / 10
    
    df_wk['week'] = int(file.split(".")[0][-1])
    df_weekly_data = pd.concat([df_weekly_data, df_wk], ignore_index=True)


  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
os.makedirs(data_dir.joinpath("standardized"), exist_ok=True)
df_weekly_data.to_parquet(data_dir.joinpath("standardized","all_weeks.parquet"), index=False)